In [2]:
import numpy as np

In [3]:
a = np.array([[1], [2], [3]])

In [11]:
a[:,0]

array([1, 2, 3])

In [7]:
y = np.array([1,2,3]).reshape(-1,1)

In [8]:
y

array([[1],
       [2],
       [3]])

In [9]:
np.array([1])

array([1])

In [10]:
np.dot(np.array([1,2,3]).reshape(-1,1), np.array([1,2,3]).reshape(1,-1))

array([[1, 2, 3],
       [2, 4, 6],
       [3, 6, 9]])

In [11]:
sc

''

In [13]:
from pyspark import SparkContext

In [14]:
sc = SparkContext()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=PySparkShell, master=local[*]) created by <module> at /Users/dirk/Virtualenvs/std/lib/python2.7/site-packages/IPython/utils/py3compat.py:289 

In [3]:
import numpy as np

In [1]:
sc

In [4]:
mu = [[0, 0], [10, 10]]
sigma = [[1, 0], [0, 1]]
points = []
labels = []
for i, m in enumerate(mu):
    points.append(np.random.multivariate_normal(m, sigma, 100))
    labels += [i]*100
points=np.vstack(points)
RDD = sc.parallelize([(x, i) for x, i in zip(points, labels)])

In [5]:
N = RDD.count()
Ns = RDD.map(lambda (_, g): (g, None)).countByKey()
print Ns

defaultdict(<type 'int'>, {0: 100, 1: 100})


In [6]:
pis = {i: Ns[i]/float(N) for i in Ns}

In [7]:
pis

{0: 0.5, 1: 0.5}

In [10]:
from operator import add

In [11]:
mus = RDD.map(lambda (x, g): (g, x)).reduceByKey(add).collect()
print mus

[(0, array([-8.733856  ,  8.01589336])), (1, array([  995.42987443,  1004.36990818]))]


In [17]:
mus = {l: mu/Ns[l] for l,mu in mus}

In [20]:
S = (RDD
             .map(lambda (x, l): np.dot((x - mus[l]).reshape(-1,1),
                                        (x - mus[l].reshape(1,-1))))
             .reduce(add))

In [22]:
(S / (200 - 2))

array([[ 1.02676682, -0.05683141],
       [-0.05683141,  1.01163928]])

In [23]:
mu = [[0, 0], [10, 10]]
sigma = [[1, 0], [0, 1]]
points = []
labels = []
N = 10000
for i, m in enumerate(mu):
    points.append(np.random.multivariate_normal(m, sigma, N))
    labels += [i]*N
points = np.vstack(points)
RDD = sc.parallelize([(x, i) for x, i in zip(points, labels)])


In [25]:
def LDA(RDD):
    '''Given a Spark RDD consisting of pairs (x, g), where x is a
    numpy array and g is an integer label, return the priors pi, the
    variance-covariance matrix Sigma, and the group means mu for
    linear discriminant analysis.

    Returns: (pi, Sigma, mu)
             where pi, mu are dictionaries containing the priors and
             centers for each class.

    Example:
    >>> mu = [[0, 0], [10, 10]]
    >>> sigma = [[1, 0], [0, 1]]
    >>> points = []
    >>> labels = []
    >>> N = 10000
    >>> for i, m in enumerate(mu):
    ...     points.append(np.random.multivariate_normal(m, sigma, N))
    ...     labels += [i]*N
    ...
    >>> points = np.vstack(points)
    >>> RDD = sc.parallelize([(x, i) for x, i in zip(points, labels)])
    >>> LDA(RDD)
    
    '''
    N = RDD.count()
    Ns = RDD.map(lambda (_, g): (g, None)).countByKey()
    pis = {i: Ns[i]/float(N) for i in Ns}
    sums = RDD.map(lambda (x, g): (g, x)).reduceByKey(add).collect()
    mus = {l: s/Ns[l] for l,s in sums}
    Sigma = (RDD
             .map(lambda (x, l): np.dot((x-mus[l]).reshape(-1,1),
                                        (x-mus[l]).reshape(1,-1)))
             .reduce(add))
    Sigma /= float(N - len(Ns))
    
    return pis, Sigma, mus


In [26]:
LDA(RDD)

({0: 0.5, 1: 0.5}, array([[ 1.0033927 ,  0.01500472],
        [ 0.01500472,  0.9900926 ]]), {0: array([-0.02056332,  0.00299521]),
  1: array([  9.99377919,  10.01389058])})

In [27]:
pis, Sigma, mus = LDA(RDD)

In [28]:
pis

{0: 0.5, 1: 0.5}

In [30]:
Sigma

array([[ 1.0033927 ,  0.01500472],
       [ 0.01500472,  0.9900926 ]])

In [31]:
mus

{0: array([-0.02056332,  0.00299521]), 1: array([  9.99377919,  10.01389058])}

In [1]:
import requests

In [5]:
response = requests.get('https://en.wikipedia.org/wiki/Representational_state_transfer')

In [8]:
import json

In [11]:
for k in response.headers:
    print k, ',', response.headers[k]

Date , Sat, 11 Mar 2017 08:48:47 GMT
Content-Type , text/html; charset=UTF-8
Content-Length , 21753
Connection , keep-alive
Server , mw1240.eqiad.wmnet
Vary , Accept-Encoding,Cookie,Authorization
X-Powered-By , HHVM/3.12.14
X-UA-Compatible , IE=Edge
Content-language , en
Content-Encoding , gzip
P3P , CP="This is not a P3P policy! See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."
X-Content-Type-Options , nosniff
Last-Modified , Tue, 07 Mar 2017 07:47:29 GMT
Backend-Timing , D=83624 t=1488912602973474
X-Varnish , 565371581 573348527, 403841898 643614795, 100811527 841270377
Via , 1.1 varnish-v4, 1.1 varnish-v4, 1.1 varnish-v4
Age , 223120
X-Cache , cp1066 hit/2, cp3043 hit/21, cp3043 hit/60
X-Cache-Status , hit
Strict-Transport-Security , max-age=31536000; includeSubDomains; preload
Set-Cookie , WMF-Last-Access=11-Mar-2017;Path=/;HttpOnly;secure;Expires=Wed, 12 Apr 2017 00:00:00 GMT, WMF-Last-Access-Global=11-Mar-2017;Path=/;Domain=.wikipedia.org;HttpOnly;sec

In [ ]:
import pa